In [4]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("/content/train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [5]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [6]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [7]:
sequences = tokenizer.texts_to_sequences(texts)

In [8]:
padded_sequences

array([[   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       [   0,    0,    0, ...,   29,    5, 3496],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]], dtype=int32)

In [9]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [10]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [11]:
one_hot_labels

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [12]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

In [13]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.3763 - loss: 1.5232 - val_accuracy: 0.7147 - val_loss: 0.8581
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 17s 31ms/step - accuracy: 0.8623 - loss: 0.4521 - val_accuracy: 0.8112 - val_loss: 0.6114
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.9803 - loss: 0.0763 - val_accuracy: 0.8213 - val_loss: 0.6136
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9935 - loss: 0.0287 - val_accuracy: 0.8119 - val_loss: 0.6940
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.9963 - loss: 0.0185 - val_accuracy: 0.8053 - val_loss: 0.7145
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.9972 - loss: 0.0148 - val_accuracy: 0.8116 - val_loss: 0.7282
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.9969 - loss: 0.0119 - val_accuracy: 0.8181 - val_loss: 0.7726
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.9974 - loss: 0.0129 - val_accurac

In [15]:

def predict(input_text):
  input_sequence = tokenizer.texts_to_sequences([input_text])
  padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
  prediction = model.predict(padded_input_sequence)
  predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
  print(predicted_label[0])
# Preprocess the input text
text = "i am going to dance tonight."
predict(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
joy
